Notebook to test OTP configuration.
Written by Vivek
Started July 7th, 2016

We need to test the different parameters like bike and walk, etc.  Build functions that test numerous locations, settings, and generate times.

Current Todos:

TODO: Geocode Addresses into GPS using Google

TODO: Push resulting data into pandas dataframe

In [94]:
import requests
import json
import pandas as pd

Sample Api Call:
http://vta2.amigocloud.com/api/otp/plan?date=2016-07-07&time=23%3A0&mode=WALK%2CBUSISH%2CTRAINISH&fromPlace=37.389268%2C-121.879746&toPlace=37.4031595%2C-121.9865994&numItineraries=3&maxWalkDistance=5000&bikeSpeed=10&walkReluctance=10&maxTransfers=5&triangleSlopeFactor=0.333&triangleSafetyFactor=0.333&triangleTimeFactor=0.334&optimize=TRIANGLE&arriveBy=false

In [95]:
visual = False

In [96]:
def query_string(visual = False):    
    base = 'http://vta2.amigocloud.com'
    api = '/api/otp/plan?'
    date = 'date=2016-07-07'
    time = '&time=23%3A0'
    mode = '&mode=WALK%2CBUSISH%2CTRAINISH'
    start = '&fromPlace=37.389268%2C-121.879746'
    dest = '&toPlace=37.4031595%2C-121.9865994'
    num_iter = '&numItineraries=3'
    walk = '&maxWalkDistance=5000&'
    bike = 'bikeSpeed=10'
    walk_reluc = '&walkReluctance=10'
    max_tran = '&maxTransfers=5'
    triangle = '&triangleSlopeFactor=0.333&triangleSafetyFactor=0.333&triangleTimeFactor=0.334&optimize=TRIANGLE'
    arriveby = '&arriveBy=false'
    if visual == True:
        api = '/planner?'
        query = base + api + date + time + mode + start + dest + num_iter + walk + bike + walk_reluc + max_tran + triangle + arriveby
    else:
        query = base + api + date + time + mode + start + dest + num_iter + walk + bike + walk_reluc + max_tran + triangle + arriveby

    return query

In [168]:
rand_starts=pd.read_csv('Rand_Starts.csv')
rand_dests=pd.read_csv('Rand_Dests.csv')

In [169]:
df = pd.DataFrame()
df['start_lat'] = rand_starts['Lat']
df['start_long'] = rand_starts['Long']
df['dest_lat'] = rand_dests['Lat']
df['dest_long'] = rand_dests['Long']

In [176]:
df.head()

,start_lat,start_long,dest_lat,dest_long
0,37.538234,-121.991302,37.356750,-122.039860
1,37.126105,-121.672784,37.280022,-121.907950
2,37.381230,-121.859742,37.356277,-122.058223
3,37.385547,-121.859570,37.415452,-122.019959
4,37.293663,-121.812420,37.257288,-121.817952


In [62]:
query = query_string()

In [74]:
query

'http://vta2.amigocloud.com/api/otp/plan?date=2016-07-07&time=23%3A0&mode=WALK%2CBUSISH%2CTRAINISH&fromPlace=37.389268%2C-121.879746&toPlace=37.4031595%2C-121.9865994&numItineraries=3&maxWalkDistance=5000&bikeSpeed=10&walkReluctance=10&maxTransfers=5&triangleSlopeFactor=0.333&triangleSafetyFactor=0.333&triangleTimeFactor=0.334&optimize=TRIANGLE&arriveBy=false'

In [63]:
r = requests.get(query)

In [ ]:
r.text

In [65]:
query_json = json.loads(r.text)

In [66]:
'http://docs.python-guide.org/en/latest/scenarios/json/'

'http://docs.python-guide.org/en/latest/scenarios/json/'

In [86]:
query_json['debugOutput']

{'pathCalculationTime': 771,
 'pathTimes': [7, 579],
 'precalculationTime': 22,
 'renderingTime': 0,
 'timedOut': False,
 'totalTime': 793}

In [68]:
times= query_json['debugOutput']['pathTimes']

In [69]:
times

[7, 579]

In [73]:
len(times)

2

In [92]:
query_viz = query_string(visual = True)

In [93]:
query_viz

'http://vta2.amigocloud.com/planner?date=2016-07-07&time=23%3A0&mode=WALK%2CBUSISH%2CTRAINISH&fromPlace=37.389268%2C-121.879746&toPlace=37.4031595%2C-121.9865994&numItineraries=3&maxWalkDistance=5000&bikeSpeed=10&walkReluctance=10&maxTransfers=5&triangleSlopeFactor=0.333&triangleSafetyFactor=0.333&triangleTimeFactor=0.334&optimize=TRIANGLE&arriveBy=false'